In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
#x,y,c,
poseName = 'Sus-sous'
Coords = np.load("RootCoordinates/"+poseName+"X.npy")
print(Coords)
print(len(Coords))

[[0.506152 0.262977 0.88776  ... 0.514375 0.731506 0.430062]
 [0.501963 0.263005 0.873103 ... 0.510207 0.734391 0.423405]
 [0.500027 0.265633 0.895276 ... 0.510179 0.73696  0.421344]
 ...
 [0.491972 0.295676 0.841173 ... 0.508301 0.715252 0.437651]
 [0.493965 0.295676 0.841101 ... 0.50826  0.717978 0.41625 ]
 [0.495896 0.293066 0.840637 ... 0.512283 0.718186 0.450082]]
870


In [3]:
#Helper Methods:
import math
def calcDistance(x1,y1,x2,y2):
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

In [4]:
#Get the Mean Skeleton's coordinates
PoseMeanX = np.load("Stats/"+poseName+"MeanX.npy")
print(PoseMeanX)
print(len(PoseMeanX))
PoseMeanY = np.load("Stats/"+poseName+"MeanY.npy")
print(PoseMeanY)
print(len(PoseMeanY))
MeanTuples = list(zip(PoseMeanX, PoseMeanY))
print(MeanTuples)

[0.49788706 0.49784446 0.46510533 0.42477946 0.45546175 0.53063966
 0.57152482 0.54103655 0.5        0.48019515 0.49387032 0.50396278
 0.51982366 0.51102963 0.50184025 0.49067404 0.50513339 0.48091038
 0.51495923 0.50087036 0.51053537 0.49948744 0.50212536 0.48808313
 0.50707196]
25
[-0.28569551 -0.35327253 -0.35365517 -0.39519287 -0.41215622 -0.35228169
 -0.39167877 -0.40845971 -0.5        -0.50047318 -0.6125443  -0.71520218
 -0.49992043 -0.61446776 -0.73100012 -0.27810411 -0.27780849 -0.29132813
 -0.29052615 -0.76481118 -0.76283453 -0.74345798 -0.75312921 -0.74081416
 -0.72559703]
25
[(0.49788705517241344, -0.2856955080459767), (0.4978444632183908, -0.3532725287356322), (0.46510532873563226, -0.35365517011494246), (0.4247794597701154, -0.3951928678160921), (0.4554617459770114, -0.4121562172413794), (0.5306396643678163, -0.3522816896551723), (0.5715248218390803, -0.3916787689655167), (0.5410365471264373, -0.4084597057471259), (0.5, -0.5), (0.4801951471264368, -0.500473179310345), (0.4

In [5]:
POSE = "Sus-sous"
allDistanceVectors = []
for sample in Coords:
    xCoords = sample[0::3]
    #print(xCoords)
    yPos = sample[1::3]
    yCoords = [-y for y in yPos] # need to make it negative to be consistent - the mean skeleton's y saved the negative coords.
    SampleTuples = list(zip(xCoords, yCoords))
    #print(SampleTuples)
    longZip = list(zip(MeanTuples, SampleTuples))
    #print(longZip)
    distanceVector = []
    for ((x1,y1),(x2,y2)) in longZip:
        d = calcDistance(x1,y1,x2,y2)
        #print(d)
        distanceVector.append(d)
    allDistanceVectors.append(distanceVector)

print(allDistanceVectors)
print(len(allDistanceVectors))
print(len(allDistanceVectors[0]))
np.save("DataForSVM/"+POSE+"DistanceVectorsPerSample.npy", allDistanceVectors)

[[0.024175192260624402, 0.009745743789540831, 0.010142714467724377, 0.02614707146233498, 0.026927335254265942, 0.007996369320948441, 0.014457880872606523, 0.02196811401857353, 0.0, 0.002611715043776886, 0.004585901148898478, 0.006956932606845582, 0.0026820482286407047, 0.005395089989144376, 0.01308503880620373, 0.018361561620481373, 0.018503054485518013, 0.016337644398136685, 0.016967549614592, 0.013972938422056162, 0.01517999869104305, 0.01752156667110819, 0.01031806936393855, 0.013124833719096769, 0.009394164039929525], [0.023053686942050306, 0.011456873555693267, 0.009302996907262977, 0.026398040957423183, 0.026556858292956734, 0.010485739286791737, 0.015477855311574126, 0.021888120028484, 0.0, 0.0013930489098708894, 0.01022396314650157, 0.008489589411276376, 0.004645779823143021, 0.008256925083785236, 0.011722842588717986, 0.02116103803448127, 0.021390844695422996, 0.017662578651038107, 0.01507236710900007, 0.015804129142229794, 0.015178573167017706, 0.012969808755437687, 0.0086700

In [6]:
#These features are mixing coordinates and distances: 100 features per sample: 75 + 25
PoseAugFeaturesX = []
for i in range(len(Coords)):
    newList = list(Coords[i])+list(allDistanceVectors[i])
    PoseAugFeaturesX.append(newList)
    
print(PoseAugFeaturesX[0])
print(len(PoseAugFeaturesX[0]))
np.save("DataForSVM/"+POSE+"AugMeanDistFeaturesX.npy", PoseAugFeaturesX)

[0.5061519999999999, 0.26297700000000007, 0.8877600000000001, 0.502069, 0.34449, 0.8644689999999999, 0.469418, 0.34447500000000003, 0.854772, 0.42443999999999993, 0.36904800000000004, 0.882853, 0.459072, 0.38547200000000004, 0.8394459999999999, 0.532736, 0.34456500000000007, 0.865166, 0.5735889999999999, 0.377369, 0.882015, 0.532751, 0.388114, 0.847406, 0.5, 0.5, 0.749097, 0.477622, 0.5000260000000001, 0.756655, 0.4938269999999999, 0.6171300000000001, 0.821257, 0.508297, 0.7206440000000001, 0.8295389999999999, 0.522504, 0.500016, 0.75521, 0.5103669999999999, 0.619822, 0.8036209999999999, 0.508203, 0.742434, 0.781431, 0.4939929999999999, 0.260045, 0.962572, 0.5102709999999999, 0.26003300000000007, 0.969619, 0.487765, 0.276498, 0.855876, 0.524466, 0.27647200000000005, 0.888386, 0.49602199999999996, 0.777916, 0.740717, 0.508293, 0.777848, 0.791194, 0.508177, 0.758673, 0.5897209999999999, 0.4960089999999999, 0.761439, 0.329739, 0.49190899999999993, 0.7533690000000001, 0.33392099999999997, 